In [1]:
from itertools import combinations

import dill as pickle
import evaluate
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from datasets import Dataset
from gensim.models.keyedvectors import KeyedVectors
from ipymarkup import show_span_line_markup
from more_itertools import chunked
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from sentence_transformers import InputExample, SentenceTransformer, losses, models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    DebertaV2ForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)

from snomed_graph import *

ImportError: dlopen(/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pyarrow/lib.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libbrotlienc.1.dylib
  Referenced from: <310957A0-6CC7-3D8C-AF52-1DB47AFCD80D> /Users/chenyuli/anaconda3/envs/py3/lib/libarrow.1400.2.0.dylib
  Reason: tried: '/Users/chenyuli/anaconda3/envs/py3/lib/libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pyarrow/libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pyarrow/../../../libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pyarrow/libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pyarrow/../../../libbrotlienc.1.dylib' (no such file), '/Users/chenyuli/anaconda3/envs/py3/lib/python3.11/site-packages/pya

In [6]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 104.5 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 153.2 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
Note: you may need to restart the kernel to use updated packages.


In [2]:
random_seed = 42  # For reproducibility
max_seq_len = 512  # Maximum sequence length for (BERT-based) encoders
cer_model_id = "microsoft/deberta-v3-large"  # Base model for Clinical Entity Recogniser
kb_embedding_model_id = ("sentence-transformers/all-MiniLM-L6-v2") # base model for concept encoder
use_LoRA = False  # Whether to use a LoRA to fine-tune the CER model

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 24.1.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.1.1



## Package Plan ##

  environment location: /Users/chenyuli/anaconda3/envs/py3

  added / updated specs:
    - evaluate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20230802.0      |       h61975a4_2         1.2 MB
    arrow-cpp-14.0.2           |       h3ade35f_1         8.5 MB
    aws-c-auth-0.6.19          |       h6c40b1e_0          87 KB
    aws-c-cal-0.5.20           |       h3333b6a_0          35 KB
    aws-c-common-0.8.5         |       h6c40b1e_0         188 KB
    aws-c-compression-0.2.16   |       h6c40b1e_0  